# Purpose
to parse the text files from the new RFID system around the hut in wedge 4

# Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os, glob, csv, sys

# User defined locations

In [2]:
listoffiles=[]
fld='/Users/dennise/Desktop/LOGGER'
for folder in os.listdir(fld):
    if len(folder) > 2:
        folder_path = os.path.join(fld,folder)
        for file_name in os.listdir(folder_path):
            listoffiles.append(os.path.join(folder_path,file_name))
listoffiles

['/Users/dennise/Desktop/LOGGER\\2022_06\\2022_06_16.log',
 '/Users/dennise/Desktop/LOGGER\\2022_06\\2022_06_17.log',
 '/Users/dennise/Desktop/LOGGER\\2022_06\\2022_06_18.log',
 '/Users/dennise/Desktop/LOGGER\\2022_06\\2022_06_19.log',
 '/Users/dennise/Desktop/LOGGER\\2022_06\\2022_06_20.log',
 '/Users/dennise/Desktop/LOGGER\\2022_06\\2022_06_21.log',
 '/Users/dennise/Desktop/LOGGER\\2022_06\\2022_06_22.log',
 '/Users/dennise/Desktop/LOGGER\\2022_06\\2022_06_23.log',
 '/Users/dennise/Desktop/LOGGER\\2022_06\\2022_06_24.log',
 '/Users/dennise/Desktop/LOGGER\\2022_06\\2022_06_25.log',
 '/Users/dennise/Desktop/LOGGER\\2022_06\\2022_06_26.log',
 '/Users/dennise/Desktop/LOGGER\\2022_06\\2022_06_27.log',
 '/Users/dennise/Desktop/LOGGER\\2022_06\\2022_06_28.log',
 '/Users/dennise/Desktop/LOGGER\\2022_06\\2022_06_29.log',
 '/Users/dennise/Desktop/LOGGER\\2022_06\\2022_06_30.log',
 '/Users/dennise/Desktop/LOGGER\\2022_07\\2022_07_01.log',
 '/Users/dennise/Desktop/LOGGER\\2022_07\\2022_07_02.log

In [3]:
txtfile=listoffiles[0]
txtfile

'/Users/dennise/Desktop/LOGGER\\2022_06\\2022_06_16.log'

# Functions

In [5]:
def get_txtfile_contents(txtfile):
    dates=[]
    first=[]
    second=[]
    third=[]
    tagvals=[]
    mvs=[]

    # import file
    f = open(txtfile, 'r') 
    for x in f.readlines():
        if "TAG" in x:
            values=[xval.split(' ') for xval in x.split(':')]
            dates.append(values[1][2])
            first.append(values[1][3])
            second.append(values[2][0])
            third.append(values[3][0])
            tagvals.append(values[3][1])
            mvs.append(values[3][2])

    # want this to be a dataframe
    df=pd.DataFrame({'dates':dates,'hour':first,'minute':second,'second':third,'tagvals':tagvals,'mvs':mvs})
    df['converted_tags'] = df.tagvals
    for i in np.arange(0,len(df.tagvals)):
        val=df.converted_tags[i]
        df.converted_tags[i]=int(val[4:],base=16)
    return df

# Exploration

In [7]:
x=-1
for txtfile in listoffiles:
    x+=1
    if x==0:
        df_alldata = get_txtfile_contents(txtfile)
    else:
        df = get_txtfile_contents(txtfile)
        df_alldata = pd.concat([df_alldata,df],ignore_index=True)
    print(x)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41


In [8]:
df_alldata.to_csv('/Users/dennise/Desktop/2022_RFID_hut_data.csv')

In [9]:
unique_tags = np.unique(df_alldata.converted_tags)
unique_tags[1:]

array([43000130206, 43000130217, 43000130223, 43000130232, 43000130236,
       43000130250, 43000130258, 43000130280, 43000130282, 43000130291,
       43000130324, 43000130337, 43000130345, 43000130355, 43000130357,
       43000130359, 43000130399, 67000063703], dtype=object)

In [ ]:
for animal in unique_tags[1:]:
    

In [37]:
for animal in unique_tags:
    subdf=df_alldata[df_alldata.converted_tags==animal]
    print(animal)
    print(list(subdf.dates)[-1])
    print('   ')

7425
07/27/2022
   
43000130206
06/17/2022
   
43000130217
06/16/2022
   
43000130223
06/26/2022
   
43000130232
07/26/2022
   
43000130236
06/17/2022
   
43000130250
07/26/2022
   
43000130258
06/16/2022
   
43000130280
06/22/2022
   
43000130282
06/17/2022
   
43000130291
07/27/2022
   
43000130324
07/26/2022
   
43000130337
07/26/2022
   
43000130345
07/26/2022
   
43000130355
07/26/2022
   
43000130357
06/16/2022
   
43000130359
06/16/2022
   
43000130399
07/26/2022
   
67000063703
07/26/2022
   


In [11]:
df_alldata

,dates,hour,minute,second,tagvals,mvs,converted_tags
0,06/16/2022,17,33,08.090,3E7.0000001D01,714,7425
1,06/16/2022,17,42,33.580,384.0A0301EAAF,498,43000130223
2,06/16/2022,17,42,33.610,384.0A0301EAAF,512,43000130223
3,06/16/2022,17,42,33.650,384.0A0301EAAF,502,43000130223
4,06/16/2022,17,42,33.680,384.0A0301EAAF,494,43000130223
...,...,...,...,...,...,...,...
87267,07/27/2022,06,24,41.130,3E7.0000001D01,739,7425
87268,07/27/2022,06,34,41.410,3E7.0000001D01,736,7425
87269,07/27/2022,06,44,41.200,3E7.0000001D01,734,7425
87270,07/27/2022,07,04,41.130,3E7.0000001D01,735,7425


In [45]:
df_ = df_alldata[df_alldata.dates=='07/05/2022']

In [46]:
df_=df_[df_.converted_tags!=7425]

In [86]:
df_to_plot=df_[df_.hour=='17']
df_to_plot

,dates,hour,minute,second,tagvals,mvs,converted_tags
23660,07/05/2022,17,56,54.390,384.0F9983D6D7,512,67000063703
23661,07/05/2022,17,56,54.420,384.0F9983D6D7,513,67000063703
23662,07/05/2022,17,56,54.450,384.0F9983D6D7,513,67000063703
23663,07/05/2022,17,56,54.510,384.0F9983D6D7,512,67000063703
23664,07/05/2022,17,56,54.540,384.0F9983D6D7,492,67000063703
23665,07/05/2022,17,56,54.570,384.0F9983D6D7,498,67000063703
23666,07/05/2022,17,56,54.600,384.0F9983D6D7,496,67000063703
23667,07/05/2022,17,56,54.630,384.0F9983D6D7,496,67000063703
23668,07/05/2022,17,56,54.660,384.0F9983D6D7,504,67000063703
23669,07/05/2022,17,56,54.690,384.0F9983D6D7,505,67000063703
